In [2]:
import os, json, re, string, requests, pprint, nltk, operator, numpy as np, math, tqdm
from time import gmtime, strftime
from functools import wraps
from time import time

In [1]:
def process_judgment(judgment):
    data_to_query = (",").join(judgment)
    r = requests.post(data=data_to_query.encode("utf-8"), url="http://localhost:9200")
    response_text = r.text
    splited_response = response_text.splitlines()
    splited_response = [" ".join(x.replace("\t", " ").replace("none", "")[1:].split(":")[:2][:1]).replace(" ", ":")
                        for x in splited_response if ":" in x]
    return splited_response

In [33]:
all_words = []

def read_file(file_path):
    print ("Processing file" + file_path)
    with open (file_path) as file:
        json_content = json.load(file)
        item_count = 0
        for item in tqdm.tqdm(json_content):
            judgment = []
            item_count += 1
            text_content = re.sub("<.*?>", "", item["textContent"])
            text_content = text_content.replace('-\n', '')
            word_content = text_content.split()
            topicSpecificPunctuation = '„”–§…«»'
            translator = str.maketrans('', '', string.punctuation+topicSpecificPunctuation)
            
            for word in word_content:
                word = word.translate(translator).lower()
                if len(word)>0:
                    judgment.append(word)
            unigrams = process_judgment(judgment)
                
            with open(file_path + "_results.txt", 'a') as out_file:
                for unigram in unigrams:
                    out_file.write(unigram + "\n")

                    
def read_all_judgments_from_2018():
    with open("../data_filtered/raport.txt", "a") as raport_file:
        for filename in tqdm.tqdm(os.listdir("../data_filtered/")):
            if not filename + "_results.txt" in os.listdir("../data_filtered/") \
                and not "json_results" in filename and not "raport.txt" in filename:
                raport_file.write(str(strftime("%Y-%m-%d %H:%M:%S", gmtime())) + " ")
                raport_file.write("Writing to file " + filename + "\n")
                raport_file.flush()
                read_file("../data_filtered/" + filename)
                raport_file.write(str(strftime("%Y-%m-%d %H:%M:%S", gmtime())) + " ")
                raport_file.write("Writing to file " + filename + "FINISHED \n")
                raport_file.flush()
        
read_all_judgments_from_2018()

100%|██████████| 23/23 [00:00<00:00, 24628.28it/s]


In [11]:
def read_list_from_file(file_path):
    with open(file_path) as file:
        content = file.read().split("\n")
        return content

In [12]:
content = []

In [14]:
 for filename in tqdm.tqdm(os.listdir("../data_filtered/")):
        if "json_result" in filename:
            content += read_list_from_file("../data_filtered/" + filename)

100%|██████████| 23/23 [00:00<00:00, 85.23it/s]


['sygn:subst', 'akt:subst', 'k:subst', '11617:num', 'pr:brev', '2:adj', 'ds:brev', '5032016:adj', 'prokuratura:subst', 'rejonowy:adj']


In [23]:
splited_response = content
# splited_response = [item for sublist in splited_response for item in sublist]


In [25]:
bigrams = nltk.bigrams(splited_response)
unigrams_frequency = nltk.FreqDist(splited_response)
bigrams_frequency = nltk.FreqDist(bigrams)

In [26]:
sorted_unigrams = sorted(unigrams_frequency.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_unigrams[:30])

[('w:prep', 115194),
 ('z:prep', 73018),
 ('na:prep', 42575),
 ('do:prep', 37865),
 ('dzień:subst', 26856),
 ('on:ppron3', 23873),
 ('sąd:subst', 22481),
 ('rok:brev', 19939),
 ('praca:subst', 18687),
 ('1:adj', 17945),
 ('ten:adj', 17865),
 ('od:prep', 16634),
 ('rok:subst', 16244),
 ('przez:prep', 15663),
 ('który:adj', 15464),
 ('o:prep', 14451),
 ('być:fin', 13991),
 ('art:subst', 12968),
 ('k:subst', 12259),
 ('sprawa:subst', 12159),
 ('za:prep', 11686),
 ('powód:subst', 11036),
 ('być:praet', 10590),
 ('to:subst', 10212),
 ('umowa:subst', 10083),
 ('złoty:brev', 9476),
 ('co:subst', 8824),
 ('a:subst', 8626),
 ('podstawa:subst', 8173),
 ('kwota:subst', 7898)]


In [27]:
sorted_bigrams = sorted(bigrams_frequency.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_bigrams[:30])

[(('z:prep', 'dzień:subst'), 12316),
 (('w:prep', 'dzień:subst'), 5441),
 (('w:prep', 'sprawa:subst'), 5189),
 (('na:prep', 'podstawa:subst'), 4707),
 (('w:prep', 'ten:adj'), 4162),
 (('ubezpieczenie:subst', 'społeczny:adj'), 3945),
 (('sąd:subst', 'rejonowy:adj'), 3598),
 (('zgodnie:adv', 'z:prep'), 3520),
 (('sąd:subst', 'okręgowy:adj'), 3460),
 (('praca:subst', 'w:prep'), 3061),
 (('do:prep', 'praca:subst'), 3006),
 (('na:prep', 'rzecz:subst'), 2851),
 (('od:prep', 'dzień:subst'), 2814),
 (('z:prep', 'tytuł:subst'), 2542),
 (('w:prep', 'zakres:subst'), 2441),
 (('sygn:subst', 'akt:subst'), 2405),
 (('w:prep', 'związek:subst'), 2291),
 (('związek:subst', 'z:prep'), 2281),
 (('do:prep', 'dzień:subst'), 2213),
 (('z:prep', 'art:subst'), 2202),
 (('ustęp:brev', '1:adj'), 2179),
 (('sąd:subst', 'wysoki:adj'), 2105),
 (('co:subst', 'do:prep'), 2094),
 (('prawo:subst', 'do:prep'), 2025),
 (('rok:brev', 'w:prep'), 2013),
 (('w:prep', 'okres:subst'), 2012),
 (('zeznanie:subst', 'świadek:subs

In [28]:
TOTAL_COUNT = len(splited_response)

In [29]:
def shannon_entrophy(word_occurences, total_words):
    sum = 0
    for x in np.nditer(word_occurences):
        if x!= 0:
            sum += (x/total_words) * math.log(x/total_words)
    return sum

def H(word_occurences):
    return shannon_entrophy(word_occurences, TOTAL_COUNT)

def calculate_contingency_table(bigram, bigram_count, total_words):
    first, second = bigram[0], bigram[1]
    first_occurence, second_occurence = unigrams_frequency[first], unigrams_frequency[second]
    '''
    |------  |---------| 
    | A,B    |B,notA   |
    |------  |---------|
    | A,notB |notA,notB|
    |------|---------|
    '''
    return np.array([
            [bigram_count, first_occurence-bigram_count],
            [second_occurence-bigram_count, total_words-first_occurence-second_occurence-bigram_count]])

def log_likeliheood_ratio(bigram_key):
    k = calculate_contingency_table(bigram_key, bigrams_frequency[bigram_key], TOTAL_COUNT)
    return 2 * np.sum(k) * (H(k) - H(k.sum(axis=0)) -H(k.sum(axis=1)))

In [30]:
log_ratios = [(key, log_likeliheood_ratio(key), value) for key, value in bigrams_frequency.items()]

In [31]:
sorted_by_log_ratios = sorted(log_ratios, key=operator.itemgetter(1), reverse=True)
sorted_by_log_ratios[:30]

[(('z:prep', 'dzień:subst'), 98379.52094307066, 12316),
 (('ubezpieczenie:subst', 'społeczny:adj'), 60458.153498147221, 3945),
 (('sąd:subst', 'rejonowy:adj'), 46824.21055800168, 3598),
 (('na:prep', 'podstawa:subst'), 45567.584704886212, 4707),
 (('sąd:subst', 'okręgowy:adj'), 45445.108017336817, 3460),
 (('sygn:subst', 'akt:subst'), 37335.628921829331, 2405),
 (('zgodnie:adv', 'z:prep'), 36709.283042933188, 3520),
 (('w:prep', 'sprawa:subst'), 35832.227748961261, 5189),
 (('organ:subst', 'rentowy:adj'), 33349.09469609931, 1950),
 (('w:prep', 'dzień:subst'), 29346.277396943824, 5441),
 (('na:prep', 'rzecz:subst'), 29138.426530990168, 2851),
 (('zeznanie:subst', 'świadek:subst'), 28675.42770106289, 2011),
 (('ustęp:brev', '1:adj'), 25611.82295967131, 2179),
 (('materiał:subst', 'dowodowy:adj'), 25334.508679081711, 1509),
 (('wiek:brev', 'zw:brev'), 25080.793470074197, 1535),
 (('sąd:subst', 'wysoki:adj'), 24873.647127569719, 2105),
 (('w:prep', 'ten:adj'), 23498.76434379541, 4162),
 ((

In [32]:
filtered_by_tags = [log_ratio for log_ratio in log_ratios 
                    if "subst" in log_ratio[0][0] and
                    ("subst" in log_ratio[0][1] or "adj" in log_ratio[0][1])]
filtered_by_tags_sorted = sorted(filtered_by_tags, key=operator.itemgetter(1), reverse=True)
filtered_by_tags_sorted[:30]

[(('ubezpieczenie:subst', 'społeczny:adj'), 60458.153498147221, 3945),
 (('sąd:subst', 'rejonowy:adj'), 46824.21055800168, 3598),
 (('sąd:subst', 'okręgowy:adj'), 45445.108017336817, 3460),
 (('sygn:subst', 'akt:subst'), 37335.628921829331, 2405),
 (('organ:subst', 'rentowy:adj'), 33349.09469609931, 1950),
 (('zeznanie:subst', 'świadek:subst'), 28675.42770106289, 2011),
 (('materiał:subst', 'dowodowy:adj'), 25334.508679081711, 1509),
 (('sąd:subst', 'wysoki:adj'), 24873.647127569719, 2105),
 (('skarb:subst', 'państwo:subst'), 20332.096118222762, 1027),
 (('wyrok:subst', 'sąd:subst'), 18168.523710309062, 1995),
 (('stan:subst', 'faktyczny:adj'), 17055.697900075509, 1194),
 (('podstawa:subst', 'art:subst'), 16694.583986694764, 1734),
 (('koszt:subst', 'proces:subst'), 15989.469213934561, 1201),
 (('zakład:subst', 'ubezpieczenie:subst'), 14494.437289815245, 1101),
 (('płatnik:subst', 'składka:subst'), 14387.135157296463, 911),
 (('działalność:subst', 'gospodarczy:adj'), 14264.546615038498